In [1]:
import pandas as pd

sales = pd.read_csv('sales.csv', header=0,encoding = 'unicode_escape')
sales['id'] = sales.index
sales.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE,id
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small,0
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small,1
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium,2
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium,3
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium,4


## Adding missing columns

In [2]:
def create_day(date):
    date_df = pd.DataFrame({"Date": pd.date_range(date, periods=1)})
    date_df["Day"] = date_df.Date.dt.day
    return date_df["Day"]

def create_week(date):
    date_df = pd.DataFrame({"Date": pd.date_range(date, periods=1)})
    date_df["Week"] = date_df.Date.dt.weekofyear
    return date_df["Week"]

def create_quarter(date):
    date_df = pd.DataFrame({"Date": pd.date_range(date, periods=1)})
    date_df["Quarter"] = date_df.Date.dt.quarter
    return date_df["Quarter"]

In [3]:
sales["Day"] = sales['ORDERDATE'].apply(create_day)
sales["Week"] = sales['ORDERDATE'].apply(create_week) 
sales["Quarter"] = sales['ORDERDATE'].apply(create_quarter)
sales.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE,id,Day,Week,Quarter
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,10022,USA,NaN,Yu,Kwai,Small,0,24,9,1
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,51100,France,EMEA,Henriot,Paul,Small,1,7,19,2
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,75508,France,EMEA,Da Cunha,Daniel,Medium,2,1,27,3
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,90003,USA,NaN,Young,Julie,Medium,3,25,35,3
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,NaN,USA,NaN,Brown,Julie,Medium,4,10,41,4


In [4]:
sales["MONTH_ID"] = sales["MONTH_ID"].map("{:02}".format)

In [5]:
sales["Day"] = sales["Day"].map("{:02}".format)

In [6]:
sales["DateKey"] = sales[["YEAR_ID","MONTH_ID","Day"]].apply(lambda x: ''.join(x.dropna().astype(str)),axis=1)
sales.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE,id,Day,Week,Quarter,DateKey
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,02,2003,...,USA,NaN,Yu,Kwai,Small,0,24,9,1,20030224
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,05,2003,...,France,EMEA,Henriot,Paul,Small,1,07,19,2,20030507
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,07,2003,...,France,EMEA,Da Cunha,Daniel,Medium,2,01,27,3,20030701
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,08,2003,...,USA,NaN,Young,Julie,Medium,3,25,35,3,20030825
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,USA,NaN,Brown,Julie,Medium,4,10,41,4,20031010


In [7]:
DimDate = sales[["DateKey","YEAR_ID","MONTH_ID","Day","Week","Quarter"]]
DimDate.columns = ["DateKey","Year","Month","Day","Week","Quarter"]
DimDate = DimDate.drop_duplicates(subset="DateKey", keep='first', inplace=False)
DimDate = DimDate.sort_values(by="DateKey", axis=0, ascending=True)

In [8]:
DimDate.head()

,DateKey,Year,Month,Day,Week,Quarter
578,20030106,2003,01,06,2,1
728,20030109,2003,01,09,2,1
475,20030110,2003,01,10,2,1
26,20030129,2003,01,29,5,1
266,20030131,2003,01,31,5,1


In [14]:
sales = sales.drop(["YEAR_ID","MONTH_ID","Day","Week","Quarter"], axis=1)

## Address Dimension

In [18]:
sales.columns

Index(['ORDERNUMBER', 'QUANTITYORDERED', 'PRICEEACH', 'ORDERLINENUMBER',
       'SALES', 'ORDERDATE', 'STATUS', 'QTR_ID', 'PRODUCTLINE', 'MSRP',
       'PRODUCTCODE', 'CUSTOMERNAME', 'PHONE', 'ADDRESSLINE1', 'ADDRESSLINE2',
       'CITY', 'STATE', 'POSTALCODE', 'COUNTRY', 'TERRITORY',
       'CONTACTLASTNAME', 'CONTACTFIRSTNAME', 'DEALSIZE', 'id', 'DateKey'],
      dtype='object')

In [35]:
sales["tempAdd"] = "ADD"
sales["AddressKey"] = sales["tempAdd"]+"-"+sales["id"].map(str)
sales = sales.drop(["tempAdd"], axis=1)

In [36]:
sales.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,PRODUCTLINE,MSRP,...,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE,id,DateKey,AddressKey
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,Motorcycles,95,...,NY,10022,USA,NaN,Yu,Kwai,Small,0,20030224,ADD-0
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,Motorcycles,95,...,NaN,51100,France,EMEA,Henriot,Paul,Small,1,20030507,ADD-1
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,Motorcycles,95,...,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium,2,20030701,ADD-2
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,Motorcycles,95,...,CA,90003,USA,NaN,Young,Julie,Medium,3,20030825,ADD-3
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,Motorcycles,95,...,CA,NaN,USA,NaN,Brown,Julie,Medium,4,20031010,ADD-4


In [37]:
sales.columns

Index(['ORDERNUMBER', 'QUANTITYORDERED', 'PRICEEACH', 'ORDERLINENUMBER',
       'SALES', 'ORDERDATE', 'STATUS', 'QTR_ID', 'PRODUCTLINE', 'MSRP',
       'PRODUCTCODE', 'CUSTOMERNAME', 'PHONE', 'ADDRESSLINE1', 'ADDRESSLINE2',
       'CITY', 'STATE', 'POSTALCODE', 'COUNTRY', 'TERRITORY',
       'CONTACTLASTNAME', 'CONTACTFIRSTNAME', 'DEALSIZE', 'id', 'DateKey',
       'AddressKey'],
      dtype='object')

In [38]:
DimAddress = sales[['AddressKey', 'POSTALCODE', 'ADDRESSLINE1', 'ADDRESSLINE2','CITY', 'STATE', 'COUNTRY', 'TERRITORY', 'CONTACTLASTNAME', 'CONTACTFIRSTNAME']]
DimAddress .columns = ['AddressKey', 'PostalCode', 'AddressLine1', 'AddressLine2','City', 'State', 'Country', 'Territory', 'ContactLastName', 'ContactFirstName']
DimAddress.head()

,AddressKey,PostalCode,AddressLine1,AddressLine2,City,State,Country,Territory,ContactLastName,ContactFirstName
0,ADD-0,10022,897 Long Airport Avenue,NaN,NYC,NY,USA,NaN,Yu,Kwai
1,ADD-1,51100,59 rue de l'Abbaye,NaN,Reims,NaN,France,EMEA,Henriot,Paul
2,ADD-2,75508,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,France,EMEA,Da Cunha,Daniel
3,ADD-3,90003,78934 Hillside Dr.,NaN,Pasadena,CA,USA,NaN,Young,Julie
4,ADD-4,NaN,7734 Strong St.,NaN,San Francisco,CA,USA,NaN,Brown,Julie


## Orders Dimension

In [41]:
sales = sales.drop(['POSTALCODE', 'ADDRESSLINE1', 'ADDRESSLINE2','CITY', 'STATE', 'COUNTRY', 'TERRITORY', 'CONTACTLASTNAME', 'CONTACTFIRSTNAME'], axis=1)

In [44]:
sales["tempOrd"] = "Ord"
sales["OrderKey"] = sales["tempOrd"]+"-"+sales["id"].map(str)
sales = sales.drop(["tempOrd"], axis=1)

In [46]:
sales.columns

Index(['ORDERNUMBER', 'QUANTITYORDERED', 'PRICEEACH', 'ORDERLINENUMBER',
       'SALES', 'ORDERDATE', 'STATUS', 'QTR_ID', 'PRODUCTLINE', 'MSRP',
       'PRODUCTCODE', 'CUSTOMERNAME', 'PHONE', 'DEALSIZE', 'id', 'DateKey',
       'AddressKey', 'OrderKey'],
      dtype='object')

In [48]:
DimOrder = sales[['OrderKey', 'ORDERDATE', 'ORDERNUMBER', 'QUANTITYORDERED', 'PRICEEACH','ORDERLINENUMBER', 'STATUS']]
DimOrder.columns = ['OrderKey', 'OrderDate', 'OrderNumber', 'QuantityOrdered', 'PriceEach','OrderLineNumber', 'Status']
DimOrder["OrderDate"] = pd.to_datetime(DimOrder['OrderDate'])
DimOrder.head()

c:\users\moelh\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,OrderKey,OrderDate,OrderNumber,QuantityOrdered,PriceEach,OrderLineNumber,Status
0,Ord-0,2003-02-24,10107,30,95.70,2,Shipped
1,Ord-1,2003-05-07,10121,34,81.35,5,Shipped
2,Ord-2,2003-07-01,10134,41,94.74,2,Shipped
3,Ord-3,2003-08-25,10145,45,83.26,6,Shipped
4,Ord-4,2003-10-10,10159,49,100.00,14,Shipped


## Customer Dimension

In [50]:
sales = sales.drop(['ORDERDATE', 'ORDERNUMBER', 'QUANTITYORDERED', 'PRICEEACH','ORDERLINENUMBER', 'STATUS'], axis=1)
sales.head()

,SALES,QTR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,DEALSIZE,id,DateKey,AddressKey,OrderKey
0,2871.00,1,Motorcycles,95,S10_1678,Land of Toys Inc.,2125557818,Small,0,20030224,ADD-0,Ord-0
1,2765.90,2,Motorcycles,95,S10_1678,Reims Collectables,26.47.1555,Small,1,20030507,ADD-1,Ord-1
2,3884.34,3,Motorcycles,95,S10_1678,Lyon Souveniers,+33 1 46 62 7555,Medium,2,20030701,ADD-2,Ord-2
3,3746.70,3,Motorcycles,95,S10_1678,Toys4GrownUps.com,6265557265,Medium,3,20030825,ADD-3,Ord-3
4,5205.27,4,Motorcycles,95,S10_1678,Corporate Gift Ideas Co.,6505551386,Medium,4,20031010,ADD-4,Ord-4


In [51]:
sales["tempCus"] = "Cus"
sales["CustomerKey"] = sales["tempCus"]+"-"+sales["id"].map(str)
sales = sales.drop(["tempCus"], axis=1)

In [52]:
sales.columns

Index(['SALES', 'QTR_ID', 'PRODUCTLINE', 'MSRP', 'PRODUCTCODE', 'CUSTOMERNAME',
       'PHONE', 'DEALSIZE', 'id', 'DateKey', 'AddressKey', 'OrderKey',
       'CustomerKey'],
      dtype='object')

In [53]:
DimCustomer = sales[['CustomerKey', 'CUSTOMERNAME', 'PHONE', 'DEALSIZE']]
DimCustomer.columns = ['CustomerKey', 'CustomerName', 'Phone', 'DealSize']
DimCustomer.head()

,CustomerKey,CustomerName,Phone,DealSize
0,Cus-0,Land of Toys Inc.,2125557818,Small
1,Cus-1,Reims Collectables,26.47.1555,Small
2,Cus-2,Lyon Souveniers,+33 1 46 62 7555,Medium
3,Cus-3,Toys4GrownUps.com,6265557265,Medium
4,Cus-4,Corporate Gift Ideas Co.,6505551386,Medium


## Product Dimension

In [54]:
sales = sales.drop(['CUSTOMERNAME', 'PHONE', 'DEALSIZE'], axis=1)
sales.head()

,SALES,QTR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,id,DateKey,AddressKey,OrderKey,CustomerKey
0,2871.00,1,Motorcycles,95,S10_1678,0,20030224,ADD-0,Ord-0,Cus-0
1,2765.90,2,Motorcycles,95,S10_1678,1,20030507,ADD-1,Ord-1,Cus-1
2,3884.34,3,Motorcycles,95,S10_1678,2,20030701,ADD-2,Ord-2,Cus-2
3,3746.70,3,Motorcycles,95,S10_1678,3,20030825,ADD-3,Ord-3,Cus-3
4,5205.27,4,Motorcycles,95,S10_1678,4,20031010,ADD-4,Ord-4,Cus-4


In [55]:
sales["tempPro"] = "Pro"
sales["ProductKey"] = sales["tempPro"]+"-"+sales["id"].map(str)
sales = sales.drop(["tempPro"], axis=1)

In [57]:
sales.head()

,SALES,QTR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,id,DateKey,AddressKey,OrderKey,CustomerKey,ProductKey
0,2871.00,1,Motorcycles,95,S10_1678,0,20030224,ADD-0,Ord-0,Cus-0,Pro-0
1,2765.90,2,Motorcycles,95,S10_1678,1,20030507,ADD-1,Ord-1,Cus-1,Pro-1
2,3884.34,3,Motorcycles,95,S10_1678,2,20030701,ADD-2,Ord-2,Cus-2,Pro-2
3,3746.70,3,Motorcycles,95,S10_1678,3,20030825,ADD-3,Ord-3,Cus-3,Pro-3
4,5205.27,4,Motorcycles,95,S10_1678,4,20031010,ADD-4,Ord-4,Cus-4,Pro-4


In [58]:
DimProducts = sales[['ProductKey', 'PRODUCTCODE', 'MSRP', 'PRODUCTLINE']]
DimProducts.columns = ['ProductKey', 'ProductCode', 'MSRP', 'ProductLine']
DimProducts.head()

,ProductKey,ProductCode,MSRP,ProductLine
0,Pro-0,S10_1678,95,Motorcycles
1,Pro-1,S10_1678,95,Motorcycles
2,Pro-2,S10_1678,95,Motorcycles
3,Pro-3,S10_1678,95,Motorcycles
4,Pro-4,S10_1678,95,Motorcycles


## Sales Fact

In [59]:
sales = sales.drop(['PRODUCTCODE', 'MSRP', 'PRODUCTLINE'], axis=1)
sales.head()

,SALES,QTR_ID,id,DateKey,AddressKey,OrderKey,CustomerKey,ProductKey
0,2871.00,1,0,20030224,ADD-0,Ord-0,Cus-0,Pro-0
1,2765.90,2,1,20030507,ADD-1,Ord-1,Cus-1,Pro-1
2,3884.34,3,2,20030701,ADD-2,Ord-2,Cus-2,Pro-2
3,3746.70,3,3,20030825,ADD-3,Ord-3,Cus-3,Pro-3
4,5205.27,4,4,20031010,ADD-4,Ord-4,Cus-4,Pro-4


In [60]:
FactSales = sales[['id','SALES','DateKey', 'AddressKey', 'OrderKey','CustomerKey','ProductKey']]
FactSales.columns = ['ID','Sales','DateKey', 'AddressKey', 'OrderKey','CustomerKey','ProductKey']
FactSales.head()

,ID,Sales,DateKey,AddressKey,OrderKey,CustomerKey,ProductKey
0,0,2871.00,20030224,ADD-0,Ord-0,Cus-0,Pro-0
1,1,2765.90,20030507,ADD-1,Ord-1,Cus-1,Pro-1
2,2,3884.34,20030701,ADD-2,Ord-2,Cus-2,Pro-2
3,3,3746.70,20030825,ADD-3,Ord-3,Cus-3,Pro-3
4,4,5205.27,20031010,ADD-4,Ord-4,Cus-4,Pro-4


## Creating DataWarehouse

In [77]:
import sqlite3

conn = sqlite3.connect('SalesDW.db')
cursor = conn.cursor()

try:
    cursor.execute('DROP TABLE IF EXISTS `FactSales` ')
except Exception as e:
    Print(e)

In [78]:
try:
    cursor.execute('''
         CREATE TABLE FactSales
         (ID          INTEGER PRIMARY KEY,
         Sales        Float DEFAULT 0,
         DateKey      Text,
         AddressKey   Text,
         OrderKey     Text,
         CustomerKey  Text,
         ProductKey   Text);''')
except Exception as e:
    print(e)

In [79]:
FactSales_list = FactSales.values.tolist()

In [80]:
cursor.executemany("INSERT INTO FactSales(ID,Sales,DateKey,AddressKey,OrderKey,CustomerKey,ProductKey) VALUES (?,?,?,?,?,?,?)", FactSales_list)
conn.commit()

In [81]:
try:
    cursor.execute('''
         CREATE TABLE DimProduct
         (ProductKey  Text PRIMARY KEY,
         ProductCode  Text,
         MSRP         Float DEFAULT 0,
         ProductLine  Text);''')
except Exception as e:
    print(e)

In [83]:
DimProducts_list = DimProducts.values.tolist()

In [84]:
cursor.executemany("INSERT INTO DimProduct(ProductKey,ProductCode,MSRP,ProductLine) VALUES (?,?,?,?)", DimProducts_list)
conn.commit()

In [85]:
try:
    cursor.execute('''
         CREATE TABLE DimCustomer
         (CustomerKey  Text PRIMARY KEY,
         CustomerName  Text,
         Phone         Text,
         DealSize      Text);''')
except Exception as e:
    print(e)

In [86]:
DimCustomer_list = DimCustomer.values.tolist()

In [87]:
cursor.executemany("INSERT INTO DimCustomer(CustomerKey,CustomerName,Phone,DealSize) VALUES (?,?,?,?)", DimCustomer_list)
conn.commit()

In [164]:
#DimOrder['OrderDate'] = DimOrder['OrderDate'].dt.strftime('%Y-%m-%d')

In [160]:
try:
    cursor.execute('''
         CREATE TABLE DimOrder
         (OrderKey         Text PRIMARY KEY,
         OrderDate         Text,
         OrderNumber       INTEGER,
         QuantityOrdered   INTEGER,
         PriceEach         REAL,
         OrderLineNumber   INTEGER,
         Status            Text);''')
except Exception as e:
    print(e)

In [161]:
DimOrder_list = DimOrder.values.tolist()

In [163]:
cursor.executemany("INSERT INTO DimOrder(OrderKey, OrderDate, OrderNumber, QuantityOrdered, PriceEach, OrderLineNumber, Status) VALUES (?,?,?,?,?,?,?)", DimOrder_list)
conn.commit()

In [165]:
try:
    cursor.execute('''
         CREATE TABLE DimAddress
         (AddressKey Text PRIMARY KEY,
         PostalCode Text,
         AddressLine1 Text,
         AddressLine2 Text,
         City   Text,
         State Text,
         Country Text,
         Territory Text,
         ContactLastName Text,
         ContactFirstName Text);''')
except Exception as e:
    print(e)

In [166]:
DimAddress_list = DimAddress.values.tolist()

In [167]:
cursor.executemany("INSERT INTO DimAddress(AddressKey,PostalCode,AddressLine1,AddressLine2,City,State,Country,Territory,ContactLastName,ContactFirstName) VALUES (?,?,?,?,?,?,?,?,?,?)", DimAddress_list)
conn.commit()